In [2]:
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------
-------------------------   FINAL COPY   ---------------------------------------------
-------------------------   FINAL COPY   ---------------------------------------------
-------------------------   FINAL COPY   ---------------------------------------------
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------

--**********************************************************************************--
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------
-------------------------  DML - Tables  ---------------------------------------------
--------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------


/*********************************************************/
/*********************************************************/
		--  DML - DIM Airports
/*********************************************************/
/*********************************************************/

IF (SELECT COUNT(*) FROM dim.airports) = 0
	BEGIN
INSERT INTO dim.airports (AIRPORT_ID,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE)

SELECT DISTINCT IATA_CODE
      ,AIRPORT
      ,CITY
      ,STATE
      ,COUNTRY
      ,LATITUDE
      ,LONGITUDE
  FROM dbo.airports

END

/*********************************************************/
/*********************************************************/
		--  DML for DIM Airlines
/*********************************************************/
/*********************************************************/

IF (SELECT COUNT(*) FROM dim.airlines) = 0
	BEGIN
INSERT INTO dim.Airlines (AIRLINE_CODE, AIRLINE)

SELECT DISTINCT AIRLINE_CODE
			, AIRLINE 
FROM dbo.airlines
END

/*********************************************************/
/*********************************************************/
		--  DML for DIM Calendar
/*********************************************************/
/*********************************************************/

-- Tried so hard on creating a 'Julian calendar' pk the same method you did in class,
--  with the variables, but I kept running into issues. I think what was happening 
--  was an issue with data type conversions. I could not bring in the 24hour clock 
-- into SSMS without it changing to varchar data type. Then going from varchar to date
-- kept giving errors. Not sure how but the below DML works as I would have hoped the 
-- variables do.

IF (SELECT COUNT(*) FROM dim.Calendar) = 0
BEGIN
	
	INSERT INTO dim.Calendar (pkCalendar, DateValue, Year, Month, Day)
		SELECT	DISTINCT CAST((DATEPART(YEAR, CAST(CONCAT([MONTH], '-', [DAY], '-',[YEAR])  as DATE))-1900)*1000 + DATEPART(DY, CAST(CONCAT([MONTH], '-', [DAY], '-',[YEAR]) as DATE)) as int)-- Attempt of Julian Key
			, CAST(CONCAT([MONTH], '-', [DAY], '-',[YEAR])  as DATE) 'DateValue'
			, YEAR
			, MONTH
			, DAY
		FROM dbo.Flight_Logs
END
;

/*********************************************************/
/*********************************************************/
		--  DML for DIM f.FLIGHT_TABLE
/*********************************************************/
/*********************************************************/

INSERT INTO f.FLIGHT_TABLE(fkAIRLINE, FKORIGIN_AIRPORT, DESTINATION_AIRPORT, Scheduled_Departure, Departure, Departure_Delay_min, Scheduled_Arrival, Arrival_time_local, ARRIVAL_DELAY_min, Flight_Length_min, Total_Time, DIVERTED, CANCELLED)
SELECT   AIRLINE_CODE 'fkAirline'
		, ORIGIN_AIRPORT 'fkOrigin_AIRPORT'
		, DESTINATION_AIRPORT 'Destination'
		, LEFT(CAST(SCHEDULED_DEPART as time), 5) 'Scheduled Departure'
		, LEFT(CAST(DEPARTURE as time), 5) 'Departure'
		, CASE
			WHEN DEPARTURE_DELAY =' '
				THEN 0
				ELSE DEPARTURE_DELAY
			END 'Depart Delay (min)'
		, LEFT(CAST(SCHEDULED_ARRIVAL as time), 5) 'Scheduled Arrival'
		, LEFT(CAST(ARRIVAL_TIME as time), 5) 'Local Arrival'
		, ARRIVAL_DELAY
		, AIR_TIME 'Flight Length'
		, ELAPSED_TIME 'Total Time'
		, DIVERTED
		, CANCELLED	

FROM dbo.Flight_Logs

: Msg 515, Level 16, State 2, Line 87
Cannot insert the value NULL into column 'fkCalendar', table 'Flights.f.FLIGHT_TABLE'; column does not allow nulls. INSERT fails.

The statement has been terminated.

Total execution time: 00:00:03.973